In [1]:
import matplotlib.pyplot as pls
import cv2
import os
from skimage.feature.texture import graycomatrix, graycoprops
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Folder Happy
folder_path_happy = "data/happy"
images_file_happy = os.listdir(folder_path_happy)

# Folder Sad
folder_path_sad = "data/sad"
images_file_sad = os.listdir(folder_path_sad)

images_file_happy.remove("happy-0974.jpg") # image tidak terbaca
images_file_sad.remove("sad-0967.jpg") # image tidak terbaca

# Concate happy & sad
images_path = images_file_happy + images_file_sad
# images_path

In [3]:
path = f"{folder_path_happy}/{images_file_happy[200]}"
image = cv2.imread(path)
image_height, image_width, num_channels = image.shape

print("Dimensi gambar:")
print("Tinggi:", image_height)
print("Lebar:", image_width)
print("Jumlah kanal:", num_channels)

Dimensi gambar:
Tinggi: 48
Lebar: 48
Jumlah kanal: 3


In [4]:
import numpy as np
import cv2

def calculate_glcm_features(image):
    # Ubah gambar menjadi grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Hitung matriks GLCM
    distances = [1]  # Jarak antar piksel untuk GLCM
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi]  # Sudut untuk GLCM
    properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
    glcm = graycomatrix(gray_image, distances, angles, symmetric=True, normed=True)
    
    # Ekstrak fitur dari matriks GLCM
    features = []
    for prop in properties:
        prop_values = graycoprops(glcm, prop)
        features.extend(prop_values.flatten())
    
    return features

# Contoh penggunaan pada setiap gambar
features_list = []


# Dimensi yang diinginkan
target_width = 48
target_height = 48

for image_path in images_path:
    try:
        if image_path[0:5] == 'happy':
            image_happy = cv2.imread(f'{folder_path_happy}/{image_path}')
            # Resize gambar ke dimensi yang diinginkan
            resized_image_happy = cv2.resize(image_happy, (target_width, target_height))
            features_happy = calculate_glcm_features(resized_image_happy)
            features_list.append(features_happy)
        else:
            image_sad = cv2.imread(f'{folder_path_sad}/{image_path}')
            resized_image_sad = cv2.resize(image_happy, (target_width, target_height))
            features_sad = calculate_glcm_features(resized_image_sad)
            features_list.append(features_sad)
    except:
        print(image_path)


In [5]:
len(features_list)

2998

In [6]:
features_list2 = np.array(features_list).flatten()
features_list2

array([13.71099291, 17.39022182, 12.65780142, ...,  0.02211653,
        0.02114374,  0.02218749])

In [8]:
kolom = 5
features_list_copy = features_list2
diss = []
corr = []
homo = []
cont = []
asm = []
energ = []

count = 0
for i in range(len(features_list_copy)):
    if count < kolom:
        diss.append(features_list_copy[count])
    elif count >= kolom and count <kolom*2:
        corr.append(features_list_copy[count])
    elif count >= kolom*2 and count<kolom*3:
        homo.append(features_list_copy[count])
    elif count>=kolom*3 and count < kolom*4:
        cont.append(features_list_copy[count])
    elif count<=kolom*4 and count<kolom*5:
        asm.append(features_list_copy[count])
    else:
        energ.append(features_list_copy[count])
    count += 1
    if count == 30:
        features_list_copy = np.delete(features_list_copy, np.s_[0:30])
        count = 0
        
allFeature = diss + corr + homo + cont + asm +energ
len(allFeature)


89940

Progres untuk mengelompokkan fitur setiap gambar agar pas dengan sudutnya
[dissimilarity, correlation, homogeneity, contrast, ASM, energy, angles, images]

In [10]:
feature = [[] for _ in range(14990)]

index = 14990
count = 0
count_itr = 1
flag = 5
angle = 0
idx = 0
img = 1
count_flag = 0

for i in range(len(allFeature)):
    feature[idx].append(allFeature[count])
    count += index
    if i == flag:
        feature[idx].append(angle)
        feature[idx].append(f"{img}")
        flag += 6
        count_flag += 1
        angle += 45
        count = count_itr
        count_itr += 1
        idx += 1
        if count_flag%5 == 0:
            angle = 0
            img += 1
feature

[[13.710992907801899,
  0.8934856868685247,
  0.12202527186103998,
  421.5496453900757,
  0.0005533903287058143,
  0.020025455670348802,
  0,
  '1'],
 [17.390221819827364,
  0.8238089103496612,
  0.0934145289141387,
  690.9728383883996,
  0.0005596777387958489,
  0.01923814337788586,
  45,
  '1'],
 [12.657801418440119,
  0.8951072362217792,
  0.13431353827529155,
  419.2003546099346,
  0.0006266779525678256,
  0.02130889244986203,
  90,
  '1'],
 [17.070167496604263,
  0.8277037529054988,
  0.10193722212542444,
  676.88637392484,
  0.0003691888612243027,
  0.019384059815306266,
  135,
  '1'],
 [13.710992907801899,
  0.8934856868685247,
  0.12202527186103998,
  421.5496453900757,
  0.00039207896358333993,
  0.020025455670348802,
  180,
  '1'],
 [10.911347517730892,
  0.9520335441182285,
  0.1309965133944088,
  304.88829787234494,
  0.0005101643843368308,
  0.0006124375753250806,
  0,
  '2'],
 [13.297419646898593,
  0.936928581560218,
  0.10695281654185641,
  399.2793119058281,
  0.001967

In [11]:
for i in feature:
    print(i)

[13.710992907801899, 0.8934856868685247, 0.12202527186103998, 421.5496453900757, 0.0005533903287058143, 0.020025455670348802, 0, '1']
[17.390221819827364, 0.8238089103496612, 0.0934145289141387, 690.9728383883996, 0.0005596777387958489, 0.01923814337788586, 45, '1']
[12.657801418440119, 0.8951072362217792, 0.13431353827529155, 419.2003546099346, 0.0006266779525678256, 0.02130889244986203, 90, '1']
[17.070167496604263, 0.8277037529054988, 0.10193722212542444, 676.88637392484, 0.0003691888612243027, 0.019384059815306266, 135, '1']
[13.710992907801899, 0.8934856868685247, 0.12202527186103998, 421.5496453900757, 0.00039207896358333993, 0.020025455670348802, 180, '1']
[10.911347517730892, 0.9520335441182285, 0.1309965133944088, 304.88829787234494, 0.0005101643843368308, 0.0006124375753250806, 0, '2']
[13.297419646898593, 0.936928581560218, 0.10695281654185641, 399.2793119058281, 0.0019673699134853157, 0.0007824878338614999, 45, '2']
[8.579787234042774, 0.9732154079787545, 0.1718089161023538

In [12]:
df = pd.DataFrame(feature, columns=['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy', 'angles', 'images'])

pivot_table_df = pd.pivot_table(df, index='images',columns='angles', values=['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy'])
pivot_table_df.head()

ASM                                            contrast  \
angles       0         45        90        135       180         0     
images                                                                 
1       0.000553  0.000560  0.000627  0.000369  0.000392  421.549645   
10      0.000469  0.000541  0.000664  0.001144  0.000369  313.003103   
100     0.000610  0.000509  0.000755  0.000390  0.000613  269.253989   
1000    0.021547  0.000581  0.000729  0.000619  0.000661  899.050089   
1001    0.025719  0.024114  0.026994  0.024884  0.025719  431.181294   

                                                           ...    energy  \
angles          45           90           135         180  ...       0     
images                                                     ...             
1        690.972838   419.200355   676.886374  421.549645  ...  0.020025   
10       508.310095   360.235372   617.449977  313.003103  ...  0.024153   
100      602.516976   419.627216   599.168855  269.253989  ...  0.020876   
1000    1776.854685  1192.775266  1464.972386  899.050089  ...  0.022187   
1001     662.375736   349.139628   712.330014  431.181294  ...  0.000434   

                                               homogeneity            \
angles       45        90        135       180         0         45    
images                                                                 
1       0.019238  0.021309  0.019384  0.020025    0.122025  0.093415   
10      0.024458  0.024321  0.022295  0.024153    0.147566  0.110158   
100     0.020283  0.021149  0.020077  0.020876    0.124583  0.101686   
1000    0.020824  0.022117  0.021144  0.022187    0.080427  0.066353   
1001    0.000489  0.000447  0.000492  0.022187    0.177341  0.135099   

                                      
angles       90        135       180  
images                                
1       0.134314  0.101937  0.122025  
10      0.141232  0.124245  0.147566  
100     0.128753  0.098711  0.124583  
1000    0.080928  0.069194  0.080427  
1001    0.173904  0.160885  0.177341  

[5 rows x 30 columns]

In [13]:
pivot_table_df['dissimilarity'].dtypes

angles
0      float64
45     float64
90     float64
135    float64
180    float64
dtype: object

In [15]:
flatenned_table = pivot_table_df.reset_index().drop('images', axis=1)
flatenned_table

C:\Users\hp\AppData\Local\Temp\ipykernel_16240\269604551.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  flatenned_table = pivot_table_df.reset_index().drop('images', axis=1)


ASM                                            contrast  \
angles       0         45        90        135       180         0     
0       0.000553  0.000560  0.000627  0.000369  0.000392  421.549645   
1       0.000469  0.000541  0.000664  0.001144  0.000369  313.003103   
2       0.000610  0.000509  0.000755  0.000390  0.000613  269.253989   
3       0.021547  0.000581  0.000729  0.000619  0.000661  899.050089   
4       0.025719  0.024114  0.026994  0.024884  0.025719  431.181294   
...          ...       ...       ...       ...       ...         ...   
2993    0.000694  0.000576  0.000637  0.025237  0.023352  405.445479   
2994    0.026342  0.023993  0.025237  0.000329  0.000412  260.847518   
2995    0.000346  0.000342  0.018498  0.018150  0.020306  603.832890   
2996    0.018594  0.018498  0.000433  0.000578  0.000450  157.926418   
2997    0.000464  0.021547  0.020817  0.024047  0.021224  422.097961   

                                                           ...    energy  \
angles          45           90           135         180  ...       0     
0        690.972838   419.200355   676.886374  421.549645  ...  0.020025   
1        508.310095   360.235372   617.449977  313.003103  ...  0.024153   
2        602.516976   419.627216   599.168855  269.253989  ...  0.020876   
3       1776.854685  1192.775266  1464.972386  899.050089  ...  0.022187   
4        662.375736   349.139628   712.330014  431.181294  ...  0.000434   
...             ...          ...          ...         ...  ...       ...   
2993     596.832956   252.361259   540.608420  405.445479  ...  0.022117   
2994     641.610231   417.713652   547.424174  260.847518  ...  0.000447   
2995    1148.232684   571.280585   874.569488  603.832890  ...  0.021144   
2996     219.150747   102.931294   252.889543  157.926418  ...  0.000492   
2997     583.083296   356.382535   722.039837  422.097961  ...  0.022187   

                                               homogeneity            \
angles       45        90        135       180         0         45    
0       0.019238  0.021309  0.019384  0.020025    0.122025  0.093415   
1       0.024458  0.024321  0.022295  0.024153    0.147566  0.110158   
2       0.020283  0.021149  0.020077  0.020876    0.124583  0.101686   
3       0.020824  0.022117  0.021144  0.022187    0.080427  0.066353   
4       0.000489  0.000447  0.000492  0.022187    0.177341  0.135099   
...          ...       ...       ...       ...         ...       ...   
2993    0.021144  0.022187  0.000434  0.000489    0.174476  0.144589   
2994    0.000492  0.022187  0.020824  0.022117    0.146788  0.079337   
2995    0.022187  0.000434  0.000489  0.000447    0.076842  0.066620   
2996    0.022187  0.020824  0.022117  0.021144    0.244890  0.193739   
2997    0.000434  0.000489  0.000447  0.000492    0.094950  0.084808   

                                      
angles       90        135       180  
0       0.134314  0.101937  0.122025  
1       0.141232  0.124245  0.147566  
2       0.128753  0.098711  0.124583  
3       0.080928  0.069194  0.080427  
4       0.173904  0.160885  0.177341  
...          ...       ...       ...  
2993    0.187782  0.142263  0.174476  
2994    0.094203  0.086797  0.146788  
2995    0.094018  0.066235  0.076842  
2996    0.252825  0.211058  0.244890  
2997    0.117103  0.065238  0.094950  

[2998 rows x 30 columns]

In [16]:
kolom = 14990

df = pd.DataFrame({"dissimilarity":allFeature[0:kolom], "correlation":allFeature[kolom:kolom*2], "homogeneity":allFeature[kolom*2:kolom*3], 
                   "contrast":allFeature[kolom*3:kolom*4], "ASM":allFeature[kolom*4:kolom*5], "energy":allFeature[kolom*5:kolom*6]})
df

,dissimilarity,correlation,homogeneity,contrast,ASM,energy
0,13.710993,0.893486,0.122025,421.549645,0.000553,0.020025
1,17.390222,0.823809,0.093415,690.972838,0.000560,0.019238
2,12.657801,0.895107,0.134314,419.200355,0.000627,0.021309
3,17.070167,0.827704,0.101937,676.886374,0.000369,0.019384
4,13.710993,0.893486,0.122025,421.549645,0.000392,0.020025
...,...,...,...,...,...,...
14985,19.348404,0.865618,0.110490,1057.434397,0.020216,0.022187
14986,26.290629,0.778419,0.079494,1736.221820,0.000370,0.020824
14987,18.801862,0.885209,0.111103,915.314273,0.000454,0.022117
14988,24.966501,0.790373,0.090291,1644.955636,0.000376,0.021144


In [17]:
## Menyiapkan data
# Folder Happy
folder_path_happy = "data/happy"
images_file_happy = os.listdir(folder_path_happy)
images_file_happy.remove("happy-0974.jpg") # image tidak terbaca

# Folder Sad
folder_path_sad = "data/sad"
images_file_sad = os.listdir(folder_path_sad)
images_file_sad.remove("sad-0967.jpg") # image tidak terbaca

X = np.array(flatenned_table)  # Masukkan fitur GLCM di sini
y = np.array([1] * len(images_file_happy) + [0] * len(images_file_sad))  # Label: 1 untuk happy, 0 untuk sad

array([1, 1, 1, ..., 0, 0, 0])

In [18]:
# Normalisasi fitur menggunakan StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [25]:
# Membagi data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

[-0.69072005 -0.68613859  0.74534525  0.63680506  0.73417594  0.84011984
  0.88277805  0.84310027  0.84496296  0.84011984 -0.62119569 -0.65987955
 -0.47786607 -0.58960342 -0.62119569  0.78091412  0.84967886  0.83395946
  0.79516119  0.78091412  0.79055918  0.88636764 -1.08589196 -1.08191244
 -1.08773724 -0.33735566 -0.50811021 -0.55349825 -0.33250204 -0.33735566]


In [20]:
# Menentukan hyperparameter
learning_rate = 0.005
epochs = 100
num_hidden_layers = 2
hidden_neurons = [64, 32]  # Jumlah neuron pada setiap hidden layer
activation_functions = ['sigmoid', 'softmax', 'tanh']  # Fungsi aktivasi untuk setiap hidden layer

In [21]:
# Membangun model
model = tf.keras.models.Sequential()

In [22]:
# Hidden layers
for i in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(hidden_neurons[i], activation=activation_functions[i]))

# Output layer
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Mengompilasi model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Melatih model
model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=1)

# Evaluasi model pada set pengujian
_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy:', accuracy)

Epoch 1/100
75/75 [==============================] - 2s 4ms/step - loss: 0.6862 - accuracy: 0.5525
Epoch 2/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6806 - accuracy: 0.5705
Epoch 3/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6746 - accuracy: 0.5884
Epoch 4/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6704 - accuracy: 0.5867
Epoch 5/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6644 - accuracy: 0.5947
Epoch 6/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6621 - accuracy: 0.5997
Epoch 7/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6608 - accuracy: 0.6026
Epoch 8/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6594 - accuracy: 0.6047
Epoch 9/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6596 - accuracy: 0.6026
Epoch 10/100
75/75 [==============================] - 0s 2ms/step - loss: 0.6590 - accuracy: 0.5984
Epoch 11/